In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from firebase import firebase
import time
import pandas as pd
import numpy as np

In [2]:
firebase = firebase.FirebaseApplication('https://hotelwishlist-86f4f.firebaseio.com/', None)

In [3]:
browser = webdriver.Chrome("/usr/local/bin/chromedriver")

In [4]:
df = pd.read_csv(r'data/city_codes.csv')

In [5]:
result = firebase.get('/info/','')
values_view = result.values()
value_iterator = iter(values_view)
first_value = next(value_iterator)
print(first_value)

{'inDate': '08/07/2020', 'location': 'Delhi', 'outDate': '09/07/2020'}


In [6]:
#location = 'Delhi'
location = first_value['location']
location = location.lower().capitalize()
locationCode = df['iata_code'][df[df['city'] == location].index.tolist()].tolist()[0]

In [7]:
#checkin = '08/07/2020' #dd/mm/yyyy
checkin = first_value['inDate']
tempDate = checkin.split('/')
checkin_url = tempDate[1] + tempDate[0] + tempDate[2]

In [9]:
#checkout = '09/07/2020' #dd/mm/yyyy
checkout = first_value['outDate']
tempDate = checkout.split('/')
checkout_url = tempDate[1] + tempDate[0] + tempDate[2]

In [10]:
browser.implicitly_wait(20)

In [11]:
url = 'https://www.makemytrip.com/hotels/hotel-listing/?checkin=' + checkin_url + '&checkout=' + checkout_url + '&locusId=CT' + locationCode + '&locusType=city&city=CT' + locationCode + '&country=IN&searchText=' + location + '&roomStayQualifier=2e0e&_uCurrency=INR&reference=hotel&type=city'

In [12]:
browser.get(url)

In [13]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    print(last_height)
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        print(new_height)
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [14]:
#scroll(browser, 5)

2016
2016


In [21]:
# Scroll the page till bottom to get full data available in the DOM.
print ("Scrolling document upto bottom ...")
for j in range(1, 10):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5);

# Find the document body and get its inner HTML for processing in BeautifulSoup parser.
body = browser.find_element_by_tag_name("body").get_attribute("innerHTML")

Scrolling document upto bottom ...


In [15]:
body = browser.find_element_by_tag_name("body").get_attribute("innerHTML")

In [18]:
print("Getting data from DOM ...")
soupBody = BeautifulSoup(body) # Parse the inner HTML using BeautifulSoup

Getting data from DOM ...


In [19]:
pHotelName = soupBody.findAll("p", {"class": "latoBlack font22 blackText appendBottom12"}) 

In [20]:
hotelsNameData=[]
for j in range(0, len(pHotelName)):
    hotelsNameData.append(pHotelName[j].text)

In [21]:
pHotelPrice = soupBody.findAll("p", {"class": "latoBlack font26 blackText appendBottom5"}) 

In [22]:
hotelsPriceData=[]
for j in range(0, len(pHotelPrice)):
    hotelsPriceData.append(pHotelPrice[j].text)

In [23]:
scraped_data = []

for i in range(len(hotelsPriceData)):

    # initialize the dictionary
    card_details = {}

    # add data to the dictionary
    card_details['hotel_name'] = hotelsNameData[i]
    card_details['room_price'] = hotelsPriceData[i]

    # append the scraped data to the list
    scraped_data.append(card_details)

# create a data frame from the list of dictionaries
dataFrame = pd.DataFrame.from_dict(scraped_data)

In [24]:
dataFrame.insert(0, 'location', location)
dataFrame.insert(0, 'index', range(0, len(dataFrame)))

In [25]:
dataFrame

,index,location,hotel_name,room_price
0,0,Delhi,Red Fox Hotel Delhi Airport,"INR2,631"
1,1,Delhi,The Piccadily,"INR4,653"
2,2,Delhi,The Park New Delhi,"INR5,499"
3,3,Delhi,Lemon Tree Premier Delhi Airport,"INR3,283"
4,4,Delhi,Hotel Arch,"INR1,667"


In [26]:
json_records = dataFrame.to_json(r'result.json',orient ='records') 

In [27]:
json_records = dataFrame.to_dict(orient ='records')
print(json_records)

[{'index': 0, 'location': 'Delhi', 'hotel_name': 'Red Fox Hotel Delhi Airport', 'room_price': 'INR2,631'}, {'index': 1, 'location': 'Delhi', 'hotel_name': 'The Piccadily', 'room_price': 'INR4,653'}, {'index': 2, 'location': 'Delhi', 'hotel_name': 'The Park New Delhi', 'room_price': 'INR5,499'}, {'index': 3, 'location': 'Delhi', 'hotel_name': 'Lemon Tree Premier Delhi Airport', 'room_price': 'INR3,283'}, {'index': 4, 'location': 'Delhi', 'hotel_name': 'Hotel Arch', 'room_price': 'INR1,667'}]


In [28]:
firebase.delete('/hotel/', '')
for record in json_records:
    print(record)
    result = firebase.post('/hotel/',record)

{'index': 0, 'location': 'Delhi', 'hotel_name': 'Red Fox Hotel Delhi Airport', 'room_price': 'INR2,631'}
{'index': 1, 'location': 'Delhi', 'hotel_name': 'The Piccadily', 'room_price': 'INR4,653'}
{'index': 2, 'location': 'Delhi', 'hotel_name': 'The Park New Delhi', 'room_price': 'INR5,499'}
{'index': 3, 'location': 'Delhi', 'hotel_name': 'Lemon Tree Premier Delhi Airport', 'room_price': 'INR3,283'}
{'index': 4, 'location': 'Delhi', 'hotel_name': 'Hotel Arch', 'room_price': 'INR1,667'}


In [29]:
result = firebase.get('/hotel/','')
print(result)

{'-MBFrY9HE7IQ4fv-o0v0': {'hotel_name': 'Red Fox Hotel Delhi Airport', 'index': 0, 'location': 'Delhi', 'room_price': 'INR2,631'}, '-MBFrYRVm_Wfy2XGTh0Z': {'hotel_name': 'The Piccadily', 'index': 1, 'location': 'Delhi', 'room_price': 'INR4,653'}, '-MBFrYifbBW1di5Un2is': {'hotel_name': 'The Park New Delhi', 'index': 2, 'location': 'Delhi', 'room_price': 'INR5,499'}, '-MBFrZ17jNFEe-R9kxc4': {'hotel_name': 'Lemon Tree Premier Delhi Airport', 'index': 3, 'location': 'Delhi', 'room_price': 'INR3,283'}, '-MBFrZLT2E1Goff1xfnB': {'hotel_name': 'Hotel Arch', 'index': 4, 'location': 'Delhi', 'room_price': 'INR1,667'}}
